# Check All Cells — Validation Notebook

## Purpose
This notebook scans the dataset and validates state-period metrics for each .MAT file. It highlights potential issues in CC/CV periods and sign errors in charge/discharge currents.

## What it checks
- CC periods ("CC Charge" / "CC Discharge") with high current variability (std_current > 0.05)  
- CV periods ("CV Charge" / "CV Discharge") with high voltage variability (std_tension > 0.05)  
- Charge periods with negative average current (avg_current < 0)  
- Discharge periods with positive average current (avg_current > 0)

## Output
Printed lines with file path, problematic status, start/end times and the offending metric value. Use these lines to locate and inspect specific MAT files.

In [ ]:
from check_cells_functions import *
import os

In [ ]:
# Load and process data
for paste in ["../data_pub"]:
    for root, dirs, files in os.walk(paste):
        for file in files:
            if file.endswith(".mat"):
                metrics = analyze_metrics_by_status_period(process_states(os.path.join(root, file)))

                # Check if CC periods have std_current > 0.05
                cc_issues = metrics[(metrics["status"] == "CC Charge") | (metrics["status"] == "CC Discharge")]
                cc_issues = cc_issues[cc_issues["std_current"] > 0.05]
                for issue in cc_issues.itertuples():
                    print(f"File: {os.path.join(root, file)}, Issue in {issue.status} from {issue.start_time} to {issue.end_time} with std_current {issue.std_current}")

                # Check if CV periods have std_tension > 0.05
                cv_issues = metrics[(metrics["status"] == "CV Charge") | (metrics["status"] == "CV Discharge")]
                cv_issues = cv_issues[cv_issues["std_tension"] > 0.05]
                for issue in cv_issues.itertuples():
                    print(f"File: {file}, Issue in {issue.status} from {issue.start_time} to {issue.end_time} with std_tension {issue.std_tension}")

                # Check if charges have avg_current < 0
                charge_issues = metrics[(metrics["status"] == "CC Charge") | (metrics["status"] == "CV Charge")]
                charge_issues = charge_issues[charge_issues["avg_current"] < 0]
                for issue in charge_issues.itertuples():
                    print(f"File: {file}, Issue in {issue.status} from {issue.start_time} to {issue.end_time} with avg_current {issue.avg_current}")

                # Check if discharges have avg_current > 0
                discharge_issues = metrics[(metrics["status"] == "CC Discharge") | (metrics["status"] == "CV Discharge")]  
                discharge_issues = discharge_issues[discharge_issues["avg_current"] > 0]
                for issue in discharge_issues.itertuples():
                    print(f"File: {file}, Issue in {issue.status} from {issue.start_time} to {issue.end_time} with avg_current {issue.avg_current}")


File: ../data_pub/CU_Calendar/CU006/BW-VTC-174_4105_CU_cal_006_BW-VTC-CAL.mat, Issue in CC Discharge from 3.00565397208436 to 25.0945428578539 with std_current 0.3543105472864953
File: ../data_pub/CU_Cyclic/CU010_cyc/BW-VTC-232_3588_CU_cyc_010_BW-VTC-CYC2.mat, Issue in CC Discharge from 65930.3787513186 to 66360.378737912 with std_current 0.327697507275951
File: BW-VTC-395_3282_CU_cyc_002_BW-VTC-SupPos.mat, Issue in CV Discharge from 9.8564889626 to 9.9919161152 with std_tension 0.07631592079418716
File: ../data_pub/CU_Cyclic/CU007_cyc/BW-VTC-232_3155_CU_cyc_007_BW-VTC-CYC2.mat, Issue in CC Discharge from 24.2464955063267 to 24.3742732873645 with std_current 0.07615900082108763
File: ../data_pub/CU_Cyclic/CU003_cyc/BW-VTC-253_1191_CU_cyc_003_BW-VTC-CYC.mat, Issue in CC Discharge from 3.0056552242 to 3.7947299735 with std_current 0.20833071072926093
File: ../data_pub/CU_Cyclic/CU003_cyc/BW-VTC-251_1193_CU_cyc_003_BW-VTC-CYC.mat, Issue in CC Discharge from 3.0056560037 to 3.2529773618 wi